## Checking automated reconstruction quality

In [1]:
import cobra
from cobra.flux_analysis import parsimonious

# Identifies blocked reactions, 1% cutoff for fraction of optimum
def blockedReactions(model):
    
    with model as m:
        blocked = cobra.flux_analysis.variability.find_blocked_reactions(m)
        nogene_blocked = []
        for rxn in blocked:
            if m.reactions.get_by_id(rxn).gene_reaction_rule == '':
                nogene_blocked.append(rxn)

    #print(str(len(blocked)) + ' total reactions are blocked')
    fraction = (float(len(blocked)) / float(len(model.reactions))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions are blocked')
    
    return blocked


# Identify potentially gapfilled reactions, checks against pFBA solution
def missingGPR(model):
    
    noGene = []
    exclude = []
    for rxn in model.reactions:
        if len(list(rxn.genes)) == 0:
            if rxn in model.boundary:
                exclude.append(rxn.id)
                continue
            else:
                noGene.append(rxn.id)
    
    solution = parsimonious.pfba(model)
    active_rxns = set([rxn.id for rxn in model.reactions if abs(solution.fluxes[rxn.id]) > 1e-5])
    active_rxns = active_rxns.difference(set(exclude))
    noGene_active = set(noGene).intersection(active_rxns)

    fraction = float(len(model.reactions)) - float(len(exclude))
    fraction = (float(len(noGene)) / fraction) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% reactions without GPRs')
    
    fraction = (float(len(noGene_active)) / float(len(active_rxns))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% of reactions used in pFBA solution have no GPR')
    
    return noGene_active


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkFreeMass(model, cytosol='cytosol'):
    
    #if len(model.compartments.keys()) == 0:
    
    free = []
    with model as m:
    
        # Close all exchanges
        for rxn in m.exchanges: rxn.lower_bound = 0.
    
        # Create demand for each reaction and optimize individually
        reset_rxn = m.reactions[0].id
        for cpd in m.metabolites: 
            if cpd.compartment == cytosol:
                demand = cobra.Reaction('demand')
                demand.bounds = (0., 1000.)
                demand.add_metabolites({cpd: -1.0})
                m.add_reactions([demand])
                m.objective = demand
                obj_val = m.slim_optimize()
                if obj_val > 1e-8: free.append(cpd.id)
                m.objective = reset_rxn
                m.remove_reactions([demand])
    
    fraction = (float(len(free)) / float(len(model.metabolites))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% metabolites are generated for free')

    return(free)


# Checks which cytosolic metabolites are generated for free (bacteria only)
def checkLostMass(model):

    cycled = []
    with model as m:
    
        # Close all exchanges
        for rxn in m.boundary: rxn.upper_bound = 0.
    
        # Create demand for each reaction and optimize individually
        for rxn in m.reactions: 
            m.objective = rxn
            if m.slim_optimize() > 1e-6:
                for cpd in rxn.products:
                    cycled.append(cpd.id)
            
    cycled = set(cycled)
    fraction = (float(len(cycled)) / float(len(model.metabolites))) * 100.
    fraction = round(fraction, 2)
    print(str(fraction) + '% metabolites are involved in potential cycles')

    return(cycled)


# Check for mass and charge balance in reactions
def checkBalance(model):
    
    with model as m:

        elements = set()
        for cpd in m.metabolites:
            try:
                elements |= set(cpd.elements.keys())
            except:
                pass
        
        massImbal = []
        failed = 0
        if len(elements) == 0:
            print('No elemental data associated with metabolites!')
            failed = 1
        else:
            for rxn in m.reactions:
                if rxn in m.boundary:
                    continue

                try:
                    test = rxn.check_mass_balance()
                except ValueError:
                    continue

                if len(list(test)) > 0:
                    if len(set(test.keys()).intersection(elements)) > 0: massImbal.append(rxn.id)
                        
    if failed != 1:
        fraction = (float(len(massImbal)) / float(len(model.reactions))) * 100.
        fraction = round(fraction, 2)
        print(str(fraction) + '% reactions are mass imbalanced')
        
    return massImbal


def basicCheck(model):
    
    # Determination
    determination = float(len(model.reactions)) / float(len(model.metabolites))
    determination = round(determination, 3)
    if len(model.reactions) < len(model.metabolites): 
        statement = ' (overdetermined)'
    elif len(model.reactions) > len(model.metabolites):
        statement = ' (underdetermined)'
    print('Reactions to metabolites ratio: ' + str(determination) + statement)
    
    # Compartments
    print('GENRE has ' + str(len(model.compartments.keys())) + ' compartment(s)')
    
    # Genes
    if len(model.genes) == 0: 
        print('GENRE has no gene data')
    else:
        print('GENRE has ' + str(len(model.genes)) + ' genes')
    no_rxns = []
    for gene in model.genes:
          if len(gene.reactions) == 0:
                no_rxns.append(gene.id)
    if len(no_rxns) > 0:
        print('\t' + str(len(no_rxns)) + ' are not associated with reactions')
        
    # Growth
    ov = model.slim_optimize(error_value=0.)
    if ov < 1e-6:
        for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        ov = model.slim_optimize(error_value=0.)
        if ov < 1e-6:
            print('GENRE cannot acheive objective flux')
        else:
            ov = round(ov, 3)
            print(str(ov) + ' objective flux, only in complete media')
    else:
        ov = round(ov, 3)
        print(str(ov) + ' objective flux in current media')

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

def count_components(model):
    
    metabolic = 0
    transporter = 0
    exchs = [x.id for x in model.boundary]
    for rxn in model.reactions:
        if rxn.id in exchs: 
            continue
        else:
            substrate_comp = set([x.compartment for x in rxn.reactants])
            product_comp = set([x.compartment for x in rxn.products])
            if substrate_comp != product_comp:
                transporter += 1
            else:
                metabolic += 1
        
    print('Exchange reactions:', len(exchs))
    print('Transport reactions:', transporter)
    print('Metabolic reactions:', metabolic, '\n')
    
    compartments = set(model.compartments.keys())
    compartment_counts = {}
    for x in compartments: 
        compartment_counts[x] = 0
    for cpd in model.metabolites:
        compartment_counts[cpd.compartment] += 1
    
    for x in compartments:
        print(x, 'metabolites:', compartment_counts[x])

def intracellular_exchanges(model, cytosol='cytosol'):
    
    intra_exch = []
    for rxn in model.boundary:
        compartments = set([x.compartment for x in rxn.metabolites])
        if cytosol in compartments:
            intra_exch.append(rxn.id)
        
    print('Intracellular exchange reactions:', len(intra_exch))
    return intra_exch


def check_genre(model):
    print(model.name)
    results_dict = {}
    
    for rxn in model.exchanges:
        rxn.bounds = (-1000., 1000.)
    
    results_dict['rmGenes'] = basicCheck(model)
    #results_dict['nogene'] = missingGPR(model)
    results_dict['noGPRblocked'] = blockedReactions(model)
    results_dict['free'] = checkFreeMass(model)
    results_dict['lost'] = checkLostMass(model)
    results_dict['massImbal'] = checkBalance(model)
    print('\n')
    
    return results_dict


def new_model_id(model, name):
    model.name = name
    name = name.split()
    genes = str(len(model.genes))
    model.id = 'i' + name[0][0].upper() + name[1][0].lower() + genes
    return model


def add_annotation(model):
    
    # Genes
    for gene in model.genes:
        gene.annotation['sbo'] = 'SBO:0000243'
        gene.annotation['kegg.genes'] = gene.id
    
    # Metabolites
    for cpd in model.metabolites: 
        cpd.annotation['sbo'] = 'SBO:0000247'
        if 'cpd' in cpd.id: cpd.annotation['seed.compound'] = cpd.id.split('_')[0]

    # Reactions
    exchs = set([x.id for x in model.exchanges])
    for rxn in model.reactions:
        if 'rxn' in rxn.id: rxn.annotation['seed.reaction'] = rxn.id.split('_')[0]
        compartments = set([x.compartment for x in list(rxn.metabolites)])
        if rxn.id in exchs:
            rxn.annotation['sbo'] = 'SBO:0000627' # exchange
        elif len(compartments) > 1:
            rxn.annotation['sbo'] = 'SBO:0000185' # transport
        else:
            rxn.annotation['sbo'] = 'SBO:0000176' # metabolic

    # Biomass reactions
    model.reactions.EX_biomass.annotation['sbo'] = 'SBO:0000632'
    biomass_ids = ['dna_rxn','rna_rxn','protein_rxn','teichoicacid_rxn','peptidoglycan_rxn','lipid_rxn','cofactor_rxn','GmPos_cellwall','rxn10088_c','GmNeg_cellwall','biomass_rxn_gp','biomass_rxn_gn']
    for x in biomass_ids:
        try:
            model.reactions.get_by_id(x).annotation['sbo'] = 'SBO:0000629'
        except:
            continue
            
    return model


In [2]:
mag_genre_dir = '/home/mjenior/Desktop/active_projects/CDI_FMT_project/fmt_metaG/MAG_blast_results/reconstructions/'

### Super donor GENREs

In [3]:
# MAGs
Erectale = cobra.io.read_sbml_model(mag_genre_dir + 'Erectale.hits.sbml')
Erectale = new_model_id(Erectale, 'Eubacterium rectale')
Erectale = add_annotation(Erectale)

Afinegoldii = cobra.io.read_sbml_model(mag_genre_dir + 'Afinegoldii.hits.sbml')
Afinegoldii = new_model_id(Afinegoldii, 'Alistipes finegoldii')
Afinegoldii = add_annotation(Afinegoldii)

PspCT06 = cobra.io.read_sbml_model(mag_genre_dir + 'PspCT06.hits.sbml')
PspCT06 = new_model_id(PspCT06, 'Parabacteroides sp. CT06')
PspCT06 = add_annotation(PspCT06)

Bvulgatus = cobra.io.read_sbml_model(mag_genre_dir + 'Bvulgatus.hits.sbml')
Bvulgatus = new_model_id(Bvulgatus, 'Bacteroides vulgatus')
Bvulgatus = add_annotation(Bvulgatus)

Ahadrus = cobra.io.read_sbml_model(mag_genre_dir + 'Ahadrus.hits.sbml')
Ahadrus = new_model_id(Ahadrus, 'Anaerostipes hadrus')
Ahadrus = add_annotation(Ahadrus)

Fprausnitzii = cobra.io.read_sbml_model(mag_genre_dir + 'Fprausnitzii.hits.sbml')
Fprausnitzii = new_model_id(Fprausnitzii, 'Faecalibacterium prausnitzii')
Fprausnitzii = add_annotation(Fprausnitzii)

CspART55 = cobra.io.read_sbml_model(mag_genre_dir + 'CspART55.hits.sbml')
CspART55 = new_model_id(CspART55, 'Coprococcus sp. ART55')
CspART55 = add_annotation(CspART55)

Bdorei = cobra.io.read_sbml_model(mag_genre_dir + 'Bdorei.hits.sbml')
Bdorei = new_model_id(Bdorei, 'Bacteroides dorei')
Bdorei = add_annotation(Bdorei)

Bpseudocatenulatum = cobra.io.read_sbml_model(mag_genre_dir + 'Bpseudocatenulatum.hits.sbml')
Bpseudocatenulatum = new_model_id(Bpseudocatenulatum, 'Bifidobacterium pseudocatenulatum')
Bpseudocatenulatum = add_annotation(Bpseudocatenulatum)

Osplanchnicus = cobra.io.read_sbml_model(mag_genre_dir + 'Osplanchnicus.hits.sbml')
Osplanchnicus = new_model_id(Osplanchnicus, 'Odoribacter splanchnicus')
Osplanchnicus = add_annotation(Osplanchnicus)

Eeligens = cobra.io.read_sbml_model(mag_genre_dir + 'Eeligens.hits.sbml')
Eeligens = new_model_id(Eeligens, 'Eubacterium eligens')
Eeligens = add_annotation(Eeligens)

Pdistasonis = cobra.io.read_sbml_model(mag_genre_dir + 'Pdistasonis.hits.sbml')
Pdistasonis = new_model_id(Pdistasonis, 'Parabacteroides distasonis')
Pdistasonis = add_annotation(Pdistasonis)

Dpiger = cobra.io.read_sbml_model(mag_genre_dir + 'Dpiger.hits.sbml')
Dpiger = new_model_id(Dpiger, 'Desulfovibrio piger')
Dpiger = add_annotation(Dpiger)


# KEGG genes
Bproducta = cobra.io.read_sbml_model(mag_genre_dir + 'blautia_producta.sbml')
Bproducta = new_model_id(Bproducta, 'Blautia product')
Bproducta = add_annotation(Bproducta)

Csporogenes = cobra.io.read_sbml_model(mag_genre_dir + 'clostridium_sporogenes.sbml')
Csporogenes = new_model_id(Csporogenes, 'Clostridium sporogenes')
Csporogenes = add_annotation(Csporogenes)

In [4]:
#Erectale_results = check_genre(Erectale)
#Afinegoldii_results = check_genre(Afinegoldii)
#PspCT06_results = check_genre(PspCT06)
#Bvulgatus_results = check_genre(Bvulgatus)
#Ahadrus_results = check_genre(Ahadrus)
#Fprausnitzii_results = check_genre(Fprausnitzii)
#CspART55_results = check_genre(CspART55)
#Bdorei_results = check_genre(Bdorei)
#Bpseudocatenulatum_results = check_genre(Bpseudocatenulatum)
#Osplanchnicus_results = check_genre(Osplanchnicus)
#Eeligens_results = check_genre(Eeligens)
#Pdistasonis_results = check_genre(Pdistasonis)
#Dpiger_results = check_genre(Dpiger)
#Bproducta_results = check_genre(Bproducta)
#Csporogenes_results = check_genre(Csporogenes)

### Normal donor GENREs

In [5]:
# MAGs
Rintestinalis = cobra.io.read_sbml_model(mag_genre_dir + 'Rintestinalis1.hits.sbml')
Rintestinalis = new_model_id(Rintestinalis, 'Roseburia intestinalis')
Rintestinalis = add_annotation(Rintestinalis)

Blongum = cobra.io.read_sbml_model(mag_genre_dir + 'Blongum1.hits.sbml')
Blongum = new_model_id(Blongum, 'Bifidobacterium longum')
Blongum = add_annotation(Blongum)

Amuciniphila = cobra.io.read_sbml_model(mag_genre_dir + 'Amuciniphila1.hits.sbml')
Amuciniphila = new_model_id(Amuciniphila, 'Akkermansia muciniphila')
Amuciniphila = add_annotation(Amuciniphila)

Bcaccae = cobra.io.read_sbml_model(mag_genre_dir + 'Bcaccae1.hits.sbml')
Bcaccae = new_model_id(Bcaccae, 'Bacteroides caccae')
Bcaccae = add_annotation(Bcaccae)

Bovatus = cobra.io.read_sbml_model(mag_genre_dir + 'Bovatus2.hits.sbml')
Bovatus = new_model_id(Bovatus, 'Bacteroides ovatus')
Bovatus = add_annotation(Bovatus)

Oscillospiraceae = cobra.io.read_sbml_model(mag_genre_dir + 'Oscillospiraceae1.hits.sbml')
Oscillospiraceae = new_model_id(Oscillospiraceae, 'Oscillospiraceae bacterium J115')
Oscillospiraceae = add_annotation(Oscillospiraceae)

Ashahii = cobra.io.read_sbml_model(mag_genre_dir + 'Ashahii1.hits.sbml')
Ashahii = new_model_id(Ashahii, 'Alistipes shahii')
Ashahii = add_annotation(Ashahii)

# KEGG genes
Btheta = cobra.io.read_sbml_model(mag_genre_dir + 'bacteroides_theta_VPI5482.sbml')
Btheta = new_model_id(Btheta, 'Bacteroides thetaiotaomicron VPI5482')
Btheta = add_annotation(Btheta)

In [6]:
#Rintestinalis_results = check_genre(Rintestinalis)
#Blongum_results = check_genre(Blongum)
#Amuciniphila_results = check_genre(Amuciniphila)
#Bcaccae_results = check_genre(Bcaccae)
#Bovatus_results = check_genre(Bovatus)
#Oscillospiraceae_results = check_genre(Oscillospiraceae)
#Ashahii_results = check_genre(Ashahii)
#Btheta_results = check_genre(Btheta)

In [7]:
# Other 

# KEGG genes
Efaecalis = cobra.io.read_sbml_model(mag_genre_dir + 'enterococcus_faecalis_V583.sbml')
Efaecalis = new_model_id(Efaecalis, 'Enterococcus faecalis V583')
Efaecalis = add_annotation(Efaecalis)

Efaecium = cobra.io.read_sbml_model(mag_genre_dir + 'enterococcus_faecium_ATCC8459.sbml')
Efaecium = new_model_id(Efaecium, 'Enterococcus faecium ATCC8459')
Efaecium = add_annotation(Efaecium)

Ecoli = cobra.io.read_sbml_model(mag_genre_dir + 'escherichia_coli_K12.sbml')
Ecoli = new_model_id(Ecoli, 'Escherichia coli K12')
Ecoli = add_annotation(Ecoli)

Bcoagulans = cobra.io.read_sbml_model(mag_genre_dir + 'bacillus_coagulans_DSM1.sbml')
Bcoagulans = new_model_id(Bcoagulans, 'Bacillus coagulans DSM1')
Bcoagulans = add_annotation(Bcoagulans)

Sthermophillus = cobra.io.read_sbml_model(mag_genre_dir + 'streptococcus_thermophillus.sbml')
Sthermophillus = new_model_id(Sthermophillus, 'Streptococcus thermophillus')
Sthermophillus = add_annotation(Sthermophillus)

Caerofaciens = cobra.io.read_sbml_model(mag_genre_dir + 'collinsella_aerofaciens.sbml')
Caerofaciens = new_model_id(Caerofaciens, 'Collinsella aerofaciens')
Caerofaciens = add_annotation(Caerofaciens)

Cscindens = cobra.io.read_sbml_model(mag_genre_dir + 'clostridium_scindens.sbml')
Cscindens = new_model_id(Cscindens, 'Clostridium scindens')
Cscindens = add_annotation(Cscindens)

In [8]:
#Efaecalis_results = check_genre(Efaecalis)
#Efaecium_results = check_genre(Efaecium)
#Ecoli_results = check_genre(Ecoli)
#Bcoagulans_results = check_genre(Bcoagulans)
#Sthermophillus_results = check_genre(Sthermophillus)
#Caerofaciens_results = check_genre(Caerofaciens)
#Cscindens_results = check_genre(Cscindens)

In [9]:
# Write final sbml files
genre_dir = '/home/mjenior/Desktop/active_projects/CDI_FMT_project/fmt_metaG/MAG_blast_results/reconstructions/final_genres/'

# Super
cobra.io.write_sbml_model(Erectale, genre_dir + 'super/' + Erectale.id + '.sbml')
cobra.io.write_sbml_model(Afinegoldii, genre_dir + 'super/' + Afinegoldii.id + '.sbml')
cobra.io.write_sbml_model(PspCT06, genre_dir + 'super/' + PspCT06.id + '.sbml')
cobra.io.write_sbml_model(Bvulgatus, genre_dir + 'super/' + Bvulgatus.id + '.sbml')
cobra.io.write_sbml_model(Ahadrus, genre_dir + 'super/' + Ahadrus.id + '.sbml')
cobra.io.write_sbml_model(Fprausnitzii, genre_dir + 'super/' + Fprausnitzii.id + '.sbml')
cobra.io.write_sbml_model(CspART55, genre_dir + 'super/' + CspART55.id + '.sbml')
cobra.io.write_sbml_model(Bdorei, genre_dir + 'super/' + Bdorei.id + '.sbml')
cobra.io.write_sbml_model(Bpseudocatenulatum, genre_dir + 'super/' + Bpseudocatenulatum.id + '.sbml')
cobra.io.write_sbml_model(Osplanchnicus, genre_dir + 'super/' + Osplanchnicus.id + '.sbml')
cobra.io.write_sbml_model(Eeligens, genre_dir + 'super/' + Eeligens.id + '.sbml')
cobra.io.write_sbml_model(Pdistasonis, genre_dir + 'super/' + Pdistasonis.id + '.sbml')
cobra.io.write_sbml_model(Dpiger, genre_dir + 'super/' + Dpiger.id + '.sbml')
cobra.io.write_sbml_model(Bproducta, genre_dir + 'super/' + Bproducta.id + '.sbml')
cobra.io.write_sbml_model(Csporogenes, genre_dir + 'super/' + Csporogenes.id + '.sbml')

# Normal
cobra.io.write_sbml_model(Rintestinalis, genre_dir + 'normal/' + Rintestinalis.id + '.sbml')
cobra.io.write_sbml_model(Blongum, genre_dir + 'normal/' + Blongum.id + '.sbml')
cobra.io.write_sbml_model(Amuciniphila, genre_dir + 'normal/' + Amuciniphila.id + '.sbml')
cobra.io.write_sbml_model(Bcaccae, genre_dir + 'normal/' + Bcaccae.id + '.sbml')
cobra.io.write_sbml_model(Bovatus, genre_dir + 'normal/' + Bovatus.id + '.sbml')
cobra.io.write_sbml_model(Oscillospiraceae, genre_dir + 'normal/' + Oscillospiraceae.id + '.sbml')
cobra.io.write_sbml_model(Ashahii, genre_dir + 'normal/' + Ashahii.id + '.sbml')
cobra.io.write_sbml_model(Btheta, genre_dir + 'normal/' + Btheta.id + '.sbml')

# Other
cobra.io.write_sbml_model(Efaecalis, genre_dir + 'other/' + Efaecalis.id + '.sbml')
cobra.io.write_sbml_model(Efaecium, genre_dir + 'other/' + Efaecium.id + '.sbml')
cobra.io.write_sbml_model(Ecoli, genre_dir + 'other/' + Ecoli.id + '.sbml')
cobra.io.write_sbml_model(Bcoagulans, genre_dir + 'other/' + Bcoagulans.id + '.sbml')
cobra.io.write_sbml_model(Sthermophillus, genre_dir + 'other/' + Sthermophillus.id + '.sbml')
cobra.io.write_sbml_model(Caerofaciens, genre_dir + 'other/' + Caerofaciens.id + '.sbml')
cobra.io.write_sbml_model(Cscindens, genre_dir + 'other/' + Cscindens.id + '.sbml')